In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
import torch, gc
import os

In [2]:
# flair.device = torch.device('cuda')
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:40.00MiB"
torch.cuda.empty_cache()

In [3]:
columns = {0: 'text', 1: 'ner'}

In [4]:
data_folder = 'corpus-extended'

In [5]:
corpus: Corpus = ColumnCorpus(data_folder, columns, train_file='train.txt', test_file='test.txt', dev_file='dev.txt')

2023-05-11 11:39:51,356 Reading data from corpus-extended
2023-05-11 11:39:51,356 Train: corpus-extended\train.txt
2023-05-11 11:39:51,356 Dev: corpus-extended\dev.txt
2023-05-11 11:39:51,356 Test: corpus-extended\test.txt


In [6]:
len(corpus.train)

2958

In [7]:
label_type = 'ner'

In [8]:
# torch.cuda.is_available()

In [9]:
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

2023-05-11 11:39:52,189 Computing label dictionary. Progress:


2958it [00:00, 62565.25it/s]

2023-05-11 11:39:52,236 Dictionary created for label 'ner' with 1 values: LOC (seen 425 times)
Dictionary with 1 tags: LOC


In [10]:
embeddings = TransformerWordEmbeddings(model='mrm8488/bert-spanish-cased-finetuned-ner', layers='-1', subtoken_pooling='first', fine_tune=True, use_context=True, model_max_length=512)

In [11]:
tagger = SequenceTagger(hidden_size=256, embeddings=embeddings, tag_dictionary=label_dict, tag_type = 'ner', use_crf=False, use_rnn=False, reproject_embeddings=False)

2023-05-11 11:39:55,880 SequenceTagger predicts: Dictionary with 5 tags: O, S-LOC, B-LOC, E-LOC, I-LOC


In [12]:
trainer = ModelTrainer(tagger, corpus)
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [13]:
gc.collect()
torch.cuda.empty_cache()
trainer.fine_tune('ner-bertSpanish-fineTuning2', learning_rate=5.0e-6, mini_batch_size=4, mini_batch_chunk_size=1, max_epochs=20)

2023-05-11 11:39:56,069 ----------------------------------------------------------------------------------------------------
2023-05-11 11:39:56,069 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31003, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(

100%|██████████| 109/109 [00:05<00:00, 21.27it/s]

2023-05-11 11:41:59,602 Evaluating as a multi-label problem: False
2023-05-11 11:41:59,612 DEV : loss 0.006225401069968939 - f1-score (micro avg)  0.76
2023-05-11 11:41:59,622 ----------------------------------------------------------------------------------------------------


2023-05-11 11:42:11,489 epoch 2 - iter 74/740 - loss 0.01368199 - time (sec): 11.87 - samples/sec: 615.20 - lr: 0.000003
2023-05-11 11:42:23,119 epoch 2 - iter 148/740 - loss 0.01031842 - time (sec): 23.50 - samples/sec: 632.51 - lr: 0.000003
2023-05-11 11:42:34,945 epoch 2 - iter 222/740 - loss 0.01105906 - time (sec): 35.32 - samples/sec: 624.18 - lr: 0.000003
2023-05-11 11:42:46,718 epoch 2 - iter 296/740 - loss 0.01303970 - time (sec): 47.10 - samples/sec: 622.93 - lr: 0.000003
2023-05-11 11:42:58,422 epoch 2 - iter 370/740 - loss 0.01282748 - time (sec): 58.80 - samples/sec: 615.18 - lr: 0.000004
2023-05-11 11:43:10,067 epoch 2 - iter 444/740 - loss 0.01149612 - time (sec): 70.44 - samples/sec: 617.12 - lr: 0.000004
2023-05-11 11:43:21,911 epoch 2 - iter 518/740 - loss 0.01205486 - time (sec): 82.29 - samples/sec: 624.81 - lr: 0.000004
2023-05-11 11:43:33,755 epoch 2 - iter 592/740 - loss 0.01351841 - time (sec): 94.13 - samples/sec: 635.70 - lr: 0.000005
2023-05-11 11:43:45,539 e

100%|██████████| 109/109 [00:05<00:00, 20.29it/s]

...

2023-05-11 12:21:08,550 Evaluating as a multi-label problem: False


2023-05-11 12:21:08,550 DEV : loss 0.00950508750975132 - f1-score (micro avg)  0.875
2023-05-11 12:21:09,070 ----------------------------------------------------------------------------------------------------
2023-05-11 12:21:09,070 Testing using last state of model ...


100%|██████████| 161/161 [00:07<00:00, 20.74it/s]

2023-05-11 12:21:16,832 Evaluating as a multi-label problem: False
2023-05-11 12:21:16,842 0.9667	0.9508	0.9587	0.9206
2023-05-11 12:21:16,842 
Results:
- F-score (micro) 0.9587
- F-score (macro) 0.9587
- Accuracy 0.9206

By class:
              precision    recall  f1-score   support

         LOC     0.9667    0.9508    0.9587        61

   micro avg     0.9667    0.9508    0.9587        61
   macro avg     0.9667    0.9508    0.9587        61
weighted avg     0.9667    0.9508    0.9587        61

2023-05-11 12:21:16,842 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9586776859504132,
 'dev_score_history': [0.76,
  0.7906976744186047,
  0.8444444444444443,
  0.8461538461538461,
  0.9166666666666666,
  0.8070175438596492,
  0.8979591836734695,
  0.8979591836734695,
  0.8148148148148148,
  0.875,
  0.8799999999999999,
  0.8400000000000001,
  0.8571428571428572,
  0.8695652173913044,
  0.8695652173913044,
  0.8400000000000001,
  0.8510638297872339,
  0.875,
  0.875,
  0.875],
 'train_loss_history': [0.6442571589953667,
  0.0136616201387218,
  0.009767910805060523,
  0.006167142361363968,
  0.0047597321708500355,
  0.0035801430018898156,
  0.002513744380047087,
  0.0012323484908934796,
  0.0010692499069052627,
  0.00042997606189612943,
  0.0005561392065093275,
  0.000886919845380431,
  0.0003853678629491798,
  0.00044764765106009637,
  0.0007387315251185995,
  0.0006032638166711998,
  0.00013087595514463065,
  0.00015481206236612983,
  3.567651127016461e-05,
  6.14914537839758e-05],
 'dev_loss_history': [0.006225401069968939,
  0.01004